In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import IPython
import IPython.display
import os

import numpy as np

import tensorflow as tf

import matplotlib.pyplot as plt
from profiling import cumTimer

In [ ]:
print(tf.__version__)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
tf.test.is_built_with_cuda()

# Import Data

In [ ]:
InputData = np.genfromtxt('Data/InputDataBenchmark.csv',delimiter=",")
t = np.genfromtxt('Data/timeBenchmark.csv',delimiter=",")
Data = np.genfromtxt('Data/OutputDataBenchmark.csv',delimiter=",")
print(f"OutData: %s, \nInputData: %s, \nTime: %s" %(np.shape(Data),np.shape(InputData),np.shape(t)))

# Split The Data

In [ ]:
n = Data.shape[1]
train_df = Data[:, 0:int(n*0.7):]
val_df = Data[:,int(n*0.7):int(n*0.9):]
test_df = Data[:,int(n*0.9):]

num_features = Data.shape[1]


# Normalize Data

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
example_ind = 4
plt.plot(t,val_df[:,example_ind])


# Data Windowing

In [ ]:
from window import WindowGenerator

In [ ]:
OUT_STEPS = 30
multi_window = WindowGenerator(input_width=30,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               batch_size = 200)

multi_window.plotexample()
multi_window

# FeedBack Model

In [ ]:
from network import FeedBack, lstm_model

In [ ]:
model =  FeedBack(units=101, out_steps=OUT_STEPS)
model(multi_window.example[0]).shape

In [ ]:
model2 = lstm_model(return_states=False)

In [ ]:
models = os.listdir("Models/")
print(models)
print(f"Models/{models[2]}")

In [ ]:
model.load_weights(f"Models/BenchmarkLSTM@800@24-01-2022-21-11.h5AR@5024-01-2022-21-30.h5")

# Plot Case

In [ ]:
%%time
predictions = multi_window.CalcCase(val_df[:,0],t,model)

In [ ]:
example_ind = 2

In [ ]:
steps = 5
predictions = multi_window.plotCase(val_df[:,example_ind],t[:multi_window.input_width+multi_window.label_width*steps],model)

In [ ]:
predictions = multi_window.plotCase(val_df[:,example_ind],t,model)

# Create Animation

In [ ]:
from alive_progress import alive_bar

In [ ]:
from pltfigure import pltfigure
def animateDataset(dataset,timesteps,name):
    predData = []
    tFinal = multi_window.input_width+multi_window.label_width*timesteps
    with alive_bar(dataset.shape[1]) as bar:
        for i in range(dataset.shape[1]):
            print("\nTest Case:{}".format(i))
            testCase = dataset[:,i]
            ts  = multi_window.CalcCase(testCase,t[:tFinal],model)
            predData.append(ts)
            bar()
        predData =np.array(predData)
    pltfigure(dataset[:tFinal,:].T,predData,range(tFinal),"Data","Prediction",name)

In [ ]:
steps = 10
animateDataset(val_df,steps,f"Graphs/Prediction@{steps}Steps.gif")